In [ ]:
# mac version
import os
import pickle
import pandas as pd
from cobra.io import read_sbml_model

def read_exchange_reactions(tsv_path):
    with open(tsv_path, 'r') as f:
        lines = f.readlines()
    return [line.strip().split('\t')[0] for line in lines[1:] if line.strip()]

def apply_env(env, model):
    for reaction_id in EXCHANGE_REACTIONS:
        if model.reactions.has_id(reaction_id):
            model.reactions.get_by_id(reaction_id).lower_bound = -1 * env[reaction_id]
    solution = model.optimize()
    fluxes = [solution.fluxes[reaction_id] if model.reactions.has_id(reaction_id) else 0.0 for reaction_id in EXCHANGE_REACTIONS]
    return solution.objective_value, fluxes

def simulate_all_envs(env_ball, models, EXCHANGE_REACTIONS):
    result_dict = {}
    for env_idx, env in env_ball.items():
        result_dict[env_idx] = {}
        for model_id, model in models.items():
            print(f"\n Simulating model {model_id} in environment {env_idx}")
            model_copy = model.copy()
            biomass, fluxes = apply_env(env, model_copy)
            result_dict[env_idx][model_id] = (biomass, fluxes)
            print(f" Biomass = {biomass}")
            nonzero_fluxes = [(rid, flux) for rid, flux in zip(EXCHANGE_REACTIONS, fluxes) if abs(flux) > 1e-6]
            if nonzero_fluxes:
                print(f" Nonzero exchange fluxes: {nonzero_fluxes[:5]}")
            else:
                print(" All exchange fluxes are 0")
    return result_dict

if __name__ == "__main__":
    # ==== 路径设置 ====
    env_ball_path = "../../models/random_medium_samples.pkl"
    models_dir = "../../models/fixed_model"
    ex_react_path = "../../models/group_medium_exchanges.tsv"
    original_result_path = "../../results/simulation_result.pkl"  
    output_result_path = "../../results/corrected_result.pkl"

    # ==== 需要修复的模型 ====
    model_ids_to_fix = [
        "MAG013_gapfilled_noO2",
        "MAG146_gapfilled_noO2",
        "MAG214_gapfilled_noO2"
    ]

    # ==== 1. 加载环境子集 ====
    with open(env_ball_path, "rb") as f:
        env_ball = pickle.load(f)
    env_subset = {k: env_ball[k] for k in list(env_ball.keys())}
    print(f" Loaded {len(env_subset)} environments.")

    # ==== 2. 加载待修复模型 ====
    models_to_fix = {}
    for model_id in model_ids_to_fix:
        model_path = os.path.join(models_dir, model_id + ".xml")
        try:
            model = read_sbml_model(model_path)
            models_to_fix[model_id] = model
            print(f" Loaded model: {model_id}")
        except Exception as e:
            print(f" Failed to load model {model_id}: {e}")

    # ==== 3. 加载反应列表 ====
    EXCHANGE_REACTIONS = read_exchange_reactions(ex_react_path)

    # ==== 4. 加载旧结果 ====
    with open(original_result_path, "rb") as f:
        original_result = pickle.load(f)
    print(f" Loaded original simulation result.")

    # ==== 5. 重新模拟待修复模型 ====
    fixed_result = simulate_all_envs(env_subset, models_to_fix, EXCHANGE_REACTIONS)
    print(f" Finished resimulation of faulty models.")

    # ==== 6. 替换旧结果中对应模型 ====
    for env_idx in env_subset:
        for model_id in model_ids_to_fix:
            if model_id in fixed_result[env_idx]:
                original_result[env_idx][model_id] = fixed_result[env_idx][model_id]

    # ==== 7. 保存新结果 ====
    with open(output_result_path, "wb") as f:
        pickle.dump(original_result, f)

    print(f"\n Corrected results saved to `{output_result_path}`.")


 Loaded 2000 environments.
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
 Loaded model: MAG013_gapfilled_noO2
 Loaded model: MAG146_gapfilled_noO2
 Loaded model: MAG214_gapfilled_noO2
 Loaded original simulation result.

 Simulating model MAG013_gapfilled_noO2 in environment 1
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp79fon9ti.lp
Reading time = 0.01 seconds
: 900 rows, 1672 columns, 7744 nonzeros
 Biomass = 11.890948725314095
 Nonzero exchange fluxes: [('EX_cpd00254_e0', -0.03334911165967053), ('EX_cpd15560_e0', -0.03334911165967053), ('EX_cpd00058_e0', -0.03334911165967053), ('EX_cpd00063_e0', -0.03334911165967053), ('EX_cpd00030_e0', -0.03334911165967053)]

 Simulating model MAG146_gapfilled_noO2 in environment 1
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp99pl5gb_.lp
Reading time = 0.01 seconds
: 1457 rows, 3546 columns

IOStream.flush timed out


 Biomass = 17.12811191714818
 Nonzero exchange fluxes: [('EX_cpd10515_e0', -0.12416611141457617), ('EX_cpd00254_e0', -0.048037152462720736), ('EX_cpd15560_e0', -0.048037152462720736), ('EX_cpd00058_e0', -0.048037152462720736), ('EX_cpd00063_e0', -0.048037152462720736)]

 Simulating model MAG146_gapfilled_noO2 in environment 43
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp1by8gxh6.lp
Reading time = 0.01 seconds
: 1457 rows, 3546 columns, 15370 nonzeros
 Biomass = 9.917582546106638
 Nonzero exchange fluxes: [('EX_cpd10515_e0', -0.08344394766920671), ('EX_cpd00254_e0', -0.0278146492230689), ('EX_cpd15560_e0', -0.02781464922306889), ('EX_cpd00069_e0', -0.01776735682695807), ('EX_cpd00058_e0', -0.027814649223068897)]

 Simulating model MAG214_gapfilled_noO2 in environment 43
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpbfvuqq8a.lp
Reading time = 0.01 seconds
: 1441 rows, 3396 columns, 14840 nonzeros
 Biomass = 9.917

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x105612710>>
Traceback (most recent call last):
  File "/Users/azddza/Library/Caches/pypoetry/virtualenvs/electromics-project-Ddk0_V8b-py3.10/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


 Biomass = 118.04866182865914
 Nonzero exchange fluxes: [('EX_cpd10515_e0', -0.9932305896377457), ('EX_cpd00254_e0', -0.3310768632125819), ('EX_cpd15560_e0', 1.0), ('EX_cpd00058_e0', -0.3310768632125819), ('EX_cpd00063_e0', -0.3310768632125819)]

 Simulating model MAG013_gapfilled_noO2 in environment 54
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpp4_04tws.lp
Reading time = 0.00 seconds
: 900 rows, 1672 columns, 7744 nonzeros
 Biomass = 43.32097828657045
 Nonzero exchange fluxes: [('EX_cpd00254_e0', -0.12149712991440376), ('EX_cpd15560_e0', -0.12149712991440376), ('EX_cpd00058_e0', -0.12149712991440376), ('EX_cpd00063_e0', -0.12149712991440376), ('EX_cpd00030_e0', -0.12149712991440376)]

 Simulating model MAG146_gapfilled_noO2 in environment 54
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpaxyxnyt0.lp
Reading time = 0.01 seconds
: 1457 rows, 3546 columns, 15370 nonzeros
 Biomass = 100.3863466547902
 Nonzero exch

In [1]:
import os
os.environ['GRB_LICENSE_FILE'] = '../../content/licenses/gurobi.lic'

In [2]:
import os
import tempfile
import pickle
import pandas as pd
import libsbml
from cobra.io import read_sbml_model

# ==== 1. SBML 安全加载函数 ====
def safe_load_model(filepath):
    doc = libsbml.readSBML(filepath)
    if doc.getNumErrors() > 0:
        print(f"SBML 错误: {filepath}")
        for i in range(doc.getNumErrors()):
            err = doc.getError(i)
            print(f" - ({err.getSeverity()}) {err.getMessage()}")
        return None

    sbml_str = libsbml.writeSBMLToString(doc)
    with tempfile.NamedTemporaryFile("w", delete=False, suffix=".xml") as tmp:
        tmp.write(sbml_str)
        tmp_path = tmp.name
    return read_sbml_model(tmp_path)

# ==== 2. 读取 exchange reaction 列表 ====
def read_exchange_reactions(tsv_path):
    with open(tsv_path, 'r') as f:
        lines = f.readlines()
    return [line.strip().split('\t')[0] for line in lines[1:] if line.strip()]

# ==== 3. 应用环境条件并优化 ====
def apply_env(env, model):
    for reaction_id in EXCHANGE_REACTIONS:
        if model.reactions.has_id(reaction_id):
            model.reactions.get_by_id(reaction_id).lower_bound = -1 * env[reaction_id]
    solution = model.optimize()
    fluxes = [solution.fluxes[reaction_id] if model.reactions.has_id(reaction_id) else 0.0 for reaction_id in EXCHANGE_REACTIONS]
    return solution.objective_value, fluxes

# ==== 4. 多模型多环境模拟 ====
def simulate_all_envs(env_ball, models, EXCHANGE_REACTIONS):
    result_dict = {}
    for env_idx, env in env_ball.items():
        result_dict[env_idx] = {}
        for model_id, model in models.items():
            print(f"\n Simulating model {model_id} in environment {env_idx}")
            model_copy = model.copy()
            biomass, fluxes = apply_env(env, model_copy)
            result_dict[env_idx][model_id] = (biomass, fluxes)
            print(f"   ➤ Biomass: {biomass}")
            nonzero_fluxes = [(rid, flux) for rid, flux in zip(EXCHANGE_REACTIONS, fluxes) if abs(flux) > 1e-6]
            if nonzero_fluxes:
                print(f" ➤ Nonzero fluxes: {nonzero_fluxes[:5]}")
            else:
                print(" ➤ All exchange fluxes = 0")
    return result_dict

# ==== 主程序入口 ====
if __name__ == "__main__":
    # ==== 路径设置 ====
    env_ball_path = "../../models/random_medium_samples.pkl"
    models_dir = "d:/Electromics-project/models/fixed_model"
    ex_react_path = "../../models/group_medium_exchanges.tsv"
    original_result_path = "../../results/simulation_result.pkl"
    output_result_path = "../../results/corrected_result.pkl"

    # ==== 待修复模型列表 ====
    model_ids_to_fix = [
        "MAG013_gapfilled_noO2",
        "MAG146_gapfilled_noO2",
        "MAG214_gapfilled_noO2"
    ]

    # ==== 1. 加载环境 ====
    with open(env_ball_path, "rb") as f:
        env_ball = pickle.load(f)
    env_subset = {k: env_ball[k] for k in list(env_ball.keys())}
    print(f"Loaded {len(env_subset)} environments.")

    # ==== 2. 安全加载指定模型 ====
    models_to_fix = {}
    for model_id in model_ids_to_fix:
        model_path = os.path.join(models_dir, model_id + ".xml")
        print(f"加载模型: {model_id}")
        model = safe_load_model(model_path)
        if model:
            models_to_fix[model_id] = model
            print(f"成功加载: {model_id}")
        else:
            print(f"加载失败: {model_id}")

    # ==== 3. 加载 exchange reactions ====
    EXCHANGE_REACTIONS = read_exchange_reactions(ex_react_path)

    # ==== 4. 载入原始模拟结果 ====
    with open(original_result_path, "rb") as f:
        original_result = pickle.load(f)
    print(" 原始结果加载完毕。")

    # ==== 5. 重新模拟目标模型 ====
    fixed_result = simulate_all_envs(env_subset, models_to_fix, EXCHANGE_REACTIONS)
    print(" 目标模型模拟完成。")

    # ==== 6. 替换旧结果 ====
    for env_idx in env_subset:
        for model_id in model_ids_to_fix:
            if model_id in fixed_result[env_idx]:
                original_result[env_idx][model_id] = fixed_result[env_idx][model_id]

    # ==== 7. 保存更新结果 ====
    with open(output_result_path, "wb") as f:
        pickle.dump(original_result, f)

    print(f"\n Corrected results saved to `{output_result_path}`.")


Loaded 2000 environments.
加载模型: MAG013_gapfilled_noO2
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2658405
Academic license 2658405 - for non-commercial use only - registered to su___@gmail.com
成功加载: MAG013_gapfilled_noO2
加载模型: MAG146_gapfilled_noO2
成功加载: MAG146_gapfilled_noO2
加载模型: MAG214_gapfilled_noO2
成功加载: MAG214_gapfilled_noO2
 原始结果加载完毕。

 Simulating model MAG013_gapfilled_noO2 in environment 1
Read LP format model from file C:\Users\wsui\AppData\Local\Temp\tmpzuh5y9xm.lp
Reading time = 0.00 seconds
: 900 rows, 1672 columns, 7744 nonzeros
   ➤ Biomass: 11.890948725314095
 ➤ Nonzero fluxes: [('EX_cpd10515_e0', -0.10004733497901158), ('EX_cpd00254_e0', -0.03334911165967053), ('EX_cpd15560_e0', -0.03334911165967053), ('EX_cpd00058_e0', -0.03334911165967053), ('EX_cpd00063_e0', -0.03334911165967053)]

 Simulating model MAG146_gapfilled_noO2 in environment 1
Read LP format model from file C:\Users\wsui\AppData\Local\Temp\tmpet8wgii_.lp
Reading time